# Анализ данных

In [14]:
from IPython.display import HTML
import pandas as pd

## Знакомство с набором данных

Далее представлено несколько колонок полученного набора данных.

In [2]:
data = pd.read_parquet("ml_test_rec_sys.parquet")
data.head()

,UserID,GameTitle,Rating,Age,Gender,Location,Device,PlayTimeOfDay,TotalPlaytimeInHours,PurchaseHistory,InvolvementLevel,UserReview,GameGenre,GameUpdateFrequency,SocialActivity,LoadingTimeInSeconds,GameSettingsPreference
0,user_1,game_1,2,50,Male,North America,Mobile,Night,48,Yes,Beginner,Negative,RPG,Frequent,Active,16,High Graphics
1,user_1,game_2,5,50,Male,North America,Mobile,Night,48,Yes,Beginner,Negative,Sports,Frequent,Low,30,High Performance
2,user_1,game_3,2,50,Male,North America,Mobile,Night,48,Yes,Beginner,Negative,Adventure,Occasional,Low,40,High Graphics
3,user_1,game_4,3,50,Male,North America,Mobile,Night,48,Yes,Beginner,Negative,RPG,Occasional,Moderate,24,High Graphics
4,user_1,game_5,5,50,Male,North America,Mobile,Night,48,Yes,Beginner,Negative,Action,Occasional,Active,50,Balanced


Не нашел в репозитории описания данных, поэтому, опишу как я интерпретировал таблицу, для того чтобы читающему было понятнее по какой причине я принял те или иные решения.

Удельная запись представляест собой пары комбинаций пользователь-игра и соответсвнно описание пользовательского опыта в той или иной игре.

### Описание колонок

Очевидно что:

- `UserID` - уникальный идентификатор пользователя;
- `GameTitle` - уникальный идетификатор игры;
- `Age` - возраст пользователя;
- `Gender` - пол пользователя;
- `PurchaseHistory` - если история покупок у этого пользователя;
- `GameGenre` - жанр рассматриваемой игры;
- `GameUpdateFrequency` - как часто обновляется данная игра;
- `GameSettingsPreference` - некоторая особенность игры.

Для некоторых колонок не до конца понятно описывают они пользователя (одинаковые в рамках пользоваля), игру (одинаковые в рамках игры) или описывают выбранную комбинацию пользователь игра (то есть потенциально могут быть уникальными вне привязки к пользователю или игре).

Так например колонка `Rating` может описывать средний рейтинг по игре или быть отражением того, что так ее оценил пользователь который сопряжен с ней в этой записи.

Для того чтобы проверить такие моменты, для каждого пользователя/игры подсчитаем сколько уникальных значений соответсвуют ему в той или иной колонке. В следующей ячейке проводится эта процедура - в результате для каждой колонки поставлено в соответсвие уникальные ли в ней значения по пользователю и по игре.

In [34]:
display(HTML("<b>По пользователю</b>"))
by_user = (
    (data.groupby("UserID").nunique() == 1)
    .all().replace({True:"Уникальное", False:"Не уникальное"})
)
by_game = (
    (data.groupby("GameTitle").nunique() == 1)
    .all().replace({True:"Уникальное", False:"Не уникальное"})
)
pd.DataFrame({
    "По пользователю" : by_user,
    "По игре" : by_game
}).loc[data.columns]

,По пользователю,По игре
UserID,NaN,Не уникальное
GameTitle,Не уникальное,NaN
Rating,Не уникальное,Не уникальное
Age,Уникальное,Не уникальное
Gender,Уникальное,Не уникальное
Location,Уникальное,Не уникальное
Device,Уникальное,Не уникальное
PlayTimeOfDay,Уникальное,Не уникальное
TotalPlaytimeInHours,Уникальное,Не уникальное
PurchaseHistory,Уникальное,Не уникальное


Можно сделать выводы про следующие колонки:

- `Rating` указывает как клиент из записи оценил соовтевующую игру;
- `Location` указывает на условно постоянный адрес пользоватля (а не то куда он играл в ту или иную игру);
- `Device` указывает на условно постоянный девайс пользователя (а не то каой девайс он исопльзовал для игры);
- `PlayTimeOfDay` указывает когда клиент предпочитает играть - уникальный по пользователю;
- `TotalPlaytimeInHours` то сколько **всего** играл пользователь во все игры, а не то соколько он играл именно в эту игру;
- `InvolvementLevel` - то насколько пользователь вовлечен в игровую платформу в целом а не в конкретную игру;
- `UserReview` - отзыв пользователя, видимо ко всей платформе в целом а не к конкретной игре;
- `SocialActivity` - не до конца понятно что это за колонка но это какая-то уникальная характеристика игры;
- `LoadingTimeInSeconds` - сколько данная игра загружалась, получается что у всех пользователей игра зпускается одинаково быстро.